In [1]:
from datetime import datetime
from functools import lru_cache
import itertools
import glob
from pathlib import Path
import re

import numpy as np
import scipy.stats as st

import pandas as pd
import xarray as xr

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from tqdm.notebook import trange, tqdm


In [2]:
import logging
logger = logging.getLogger()

In [3]:
npPath = Path('/sharedData/scratch/all_npy3/')
ncPath = Path('/sharedData/scratch/april_data/')
acmPath = Path('/sharedData/scratch/all_npy3/')
DATAPATH = Path('/sharedData/scratch/')

In [4]:
def data2BT(rad, planck_fk1, planck_fk2, planck_bc1, planck_bc2):
    """Radiances to Brightness Temprature (using black body equation)"""
    invRad = np.array(rad)**(-1)
    arg = (invRad*planck_fk1) + 1.0
    T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2) 
    return T

In [5]:
def file2BT(npFile, ncFile):
    imageBox = np.load(npFile)   
    myFile = xr.open_dataset(ncFile)
    planck_fk1 = myFile['planck_fk1'].data
    planck_fk2 = myFile['planck_fk2'].data
    planck_bc1 = myFile['planck_bc1'].data                       
    planck_bc2 = myFile['planck_bc2'].data  
    return data2BT(imageBox.ravel(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)

In [6]:
def jointplot(G17, G16, saveout):
    g = sns.jointplot(x= G17.squeeze(), y=G16.squeeze(), kind="reg")
    g.ax_joint.set_xlabel('G17')
    g.ax_joint.set_ylabel('G16')
    g.savefig(saveout)
    plt.close()
    return

In [7]:
def validation_plot(G17, G16, _G17n, _B, AUC, savepath):
    fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize = (15,5))
    _ = ax1.plot(_G17n,_B, label=f"AUC: {AUC}")
    _ = ax1.set_ylabel('G16')
    _ = ax1.set_xlabel('G17')
    _ = ax1.set_title('Transform of G17 to G16')
    _ = ax1.set_aspect('equal')
    _ = ax1.legend(loc = 'upper left')

    _ = ax2.scatter(G17, G16)
    _ = ax2.set_ylabel('G16')
    _ = ax2.set_xlabel('G17')
    _ = ax2.set_title(f'G17 vs. G16 @{tSplice}')
    _ = ax2.set_aspect('equal')

    #_ = ax3.hist(G17, label='G17', histtype='step')
    #_ = ax3.hist(G16, label='G16', histtype='step')
    xticks = [st.scoreatpercentile(np.arange(len(G17)), p) for p in [0, 25, 50, 75, 100]]
    _ = ax3.plot(sorted(G17), label='G17')
    _ = ax3.plot(sorted(G16), label='G16')
    _ = ax3.legend(loc = 'upper left')
    _ = ax3.set_title(f'Empirical CDF for Apr-{day}-2019')
    _ = ax3.set_xticks(xticks)
    _ = ax3.set_xticklabels(['0%', '25%', '50%', '75%', '100%'])
    _ = ax3.set_xlabel('quantiles')
    _ = ax3.set_ylabel('Temperature')

    fig.savefig(savepath)
    plt.close()
    return
        

In [8]:
index = pd.date_range(start='04/08/2019', end='04/18/2019', freq='10T')
ind = index.strftime("%Y%j%H%M")
chart = pd.Series(data=None)
ctr = 0
day = 8
SS = '17vG16'
BB = str(8).zfill(2)
times_of_interest = ['0300','0910','1010','1350','1510','2100']

In [9]:
outfolder = Path("validation_figures")
outfolder.mkdir(parents=True, exist_ok=True)
joint_out = Path(outfolder, "jointplots")
joint_out.mkdir(parents=True, exist_ok=True)
transforms_out = Path(outfolder, "transforms")
transforms_out.mkdir(parents=True, exist_ok=True)
auc_out = Path(outfolder, "AUC")
auc_out.mkdir(parents=True, exist_ok=True)

In [10]:
def getfile(path, lookup):
    fpath = [f for f in path.glob(lookup)]
    if fpath:
        return fpath[0]
    return None

def extract_T16T17(tSplice, BB, ncPath, npPath):    
    lookup16 = f'*{BB}_G16_s{tSplice}*'
    ncf16 = getfile(ncPath, lookup16) 
    npf16 = getfile(npPath, lookup16)

    lookup17 = f'*{BB}_G17_s{tSplice}*'
    ncf17 = getfile(ncPath, lookup17)
    npf17 = getfile(npPath, lookup17)

    if not all([ncf16, npf16, ncf17, npf17]):
        raise ValueError(f'{tSplice}: Missing Files')
    
    T16 = file2BT(npf16, ncf16)
    T17 = file2BT(npf17, ncf17)
    if not (np.isfinite(T16).any() and np.isfinite(T17).any()):
        raise ValueError(f'{tSplice}: file is all nans')

    return T16, T17
        

In [11]:
def make_mapping(T16, T17):
    rc= np.vstack([T16, T17])
    XY = rc[:,np.isfinite(rc).all(axis=0)]
    G16 = XY[0]
    G17 = XY[1]

    _G17 = np.linspace(G17.min(), G17.max(), 1000)
    B = np.array([st.scoreatpercentile(G16, st.percentileofscore(G17,g17s,
        kind='strict')) for g17s in _G17])
    return G16, G17, _G17, B

In [12]:
def norm(D):
    return (D-D.min())/(D.max()-D.min())

In [13]:
restart = ind.str.match('20191021350').nonzero()[0][0]
restart = 0

In [14]:
SKIP = False
for ts, tSplice in tqdm(zip(index[restart:], ind[restart:]), total=index.shape[0]):
    if SKIP and (tSplice[-4:] not in times_of_interest):
        continue    
    try:
        T16, T17 = extract_T16T17(tSplice, BB, ncPath, npPath)   
    except ValueError as e:
        logger.exception(e)
        continue
    G16, G17, _G17, B = make_mapping(T16, T17)
    #print(f'{tSplice}: 16, 17, Shape: {T16.shape}, {T17.shape}')
    
    AUC = norm(B).sum()/len(B)
    chart[ts] = AUC
    #print(f'{tSplice}: AUC={AUC}')
    
    fout = f"G_{SS}_band{BB}_04-{str(day).zfill(2)}-2019_{tSplice}.png"
    
    if tSplice[-4:] in times_of_interest: 
        #make figures
        if not (joint_out/fout).exists():
            jointplot(G17, G16, joint_out/fout)
            logger.info(f'{tSplice}: joint plot')
            
        if not (transforms_out/fout).exists():
            validation_plot(G17, G16, norm(_G17), norm(B), AUC, transforms_out/fout)
            logger.info(f'{tSplice}: validation plot')
            
    if ts.day != day:
        if not (auc_out/fout).exists():
            fig2, ax0 = plt.subplots()
            chart.plot(ax=ax0)
            fig2.savefig(auc_out/fout)
            plt.close()
            logger.info(f"plotted AUC {ts.month}/{day}")
        chart.to_csv(auc_out/f'{AUC}_{ts.month}_{day}.csv', header=True)
        day+=1
        chart = pd.Series(data=None)
    
          

<ipython-input-4-078427a3f3c4>:5: RuntimeWarning: invalid value encountered in log
  T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2)
<ipython-input-4-078427a3f3c4>:5: RuntimeWarning: invalid value encountered in log
  T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2)
<ipython-input-4-078427a3f3c4>:5: RuntimeWarning: invalid value encountered in log
  T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2)
<ipython-input-4-078427a3f3c4>:5: RuntimeWarning: invalid value encountered in log
  T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2)
<ipython-input-4-078427a3f3c4>:5: RuntimeWarning: invalid value encountered in log
  T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2)
<ipython-input-4-078427a3f3c4>:5: RuntimeWarning: invalid value encountered in log
  T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2)
<ipython-input-4-078427a3f3c4>:5: RuntimeWarning: invalid value encoun